## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

July 7 July 13


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,July-10-2025,Condo/Co-op,9201 Collins Ave #1126,Surfside,FL,33154.0,1055000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9201-Collin...,MARMLS,A11833222,N,Y,25.880008,-80.121576


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-07-08    22
2025-07-11    19
2025-07-10    18
2025-07-09    15
2025-07-12     1
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
33,PAST SALE,2025-07-10,Condo/Co-op,8955 Collins Ave #702,Surfside,FL,33154.0,12250000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8955-Collin...,MARMLS,A11787450,N,Y,25.875917,-80.121106
82,PAST SALE,2025-07-11,Condo/Co-op,17975 Collins Ave #3102,Sunny Isles Beach,FL,33160.0,8900000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11735146,N,Y,25.942332,-80.120166
94,PAST SALE,2025-07-10,Condo/Co-op,10201 Collins Ave #2306,Bal Harbour,FL,33154.0,6575000.0,2.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11678342,N,Y,25.895168,-80.123150
47,PAST SALE,2025-07-10,Condo/Co-op,300 Sunny Isles Blvd Unit 4-502,Sunny Isles Beach,FL,33160.0,4700000.0,6.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11787264,N,Y,25.929383,-80.125611
81,PAST SALE,2025-07-09,Condo/Co-op,17141 Collins Ave #701,Sunny Isles Beach,FL,33160.0,4700000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11744934,N,Y,25.934939,-80.120918
55,PAST SALE,2025-07-11,Condo/Co-op,445 Grand Bay Dr #1210,Key Biscayne,FL,33149.0,4500000.0,3.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/445-Gra...,MARMLS,A11759960,N,Y,25.692619,-80.158994
19,PAST SALE,2025-07-08,Condo/Co-op,500 Alton #4304,Miami Beach,FL,33139.0,3890000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11765026,N,Y,25.775127,-80.141257
20,PAST SALE,2025-07-10,Condo/Co-op,9801 Collins Ave Unit 6A Oceanfront,Bal Harbour,FL,33154.0,2375000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11794859,N,Y,25.890231,-80.122819
8,PAST SALE,2025-07-08,Condo/Co-op,2831 S Bayshore Dr #1701,Miami,FL,33133.0,2350000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2831-S-Bayshor...,MARMLS,A11810464,N,Y,25.729356,-80.238554
99,PAST SALE,2025-07-10,Condo/Co-op,15811 Collins Ave #3302,Sunny Isles Beach,FL,33160.0,1900000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11678641,N,Y,25.922834,-80.121864


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)
        time.sleep(0.2)
        response.raise_for_status()

        # If .json() returns None, fall back on empty dict
        data = response.json() or {}

        # Drill down with or {} at each step to avoid None.get errors
        panel_info = (data.get('data') or {}) \
            .get('mainHouseInfoPanelInfo') or {}
        main_info = panel_info.get('mainHouseInfo') or {}

        # Now safe to pull lists
        listing_agents = main_info.get('listingAgents') or []
        buying_agents  = main_info.get('buyingAgents')  or []

        # Defaults
        list_agent_name_1 = list_broker_name_1 = None
        list_agent_name_2 = list_broker_name_2 = None
        buy_agent_name_1  = buy_broker_name_1  = None
        buy_agent_name_2  = buy_broker_name_2  = None

        # Extract up to two listing agents
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}) \
                                             .get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')

        # Extract up to two buying agents
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}) \
                                            .get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }


In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

## Current Week's Values

In [19]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

75


In [20]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,133,152


In [21]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$648


In [22]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$84,986,400


In [23]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:


 Miami-Dade


Input Previous Week Condo Sales Total (number units sold):
68
-----------
Input Previous Week Condo Average Sales Price:
700760
-----------
Input Previous Week Condo Average PSF:
495
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
47651700.0


In [25]:
last_week

,Condo Sales,Average Sale Price,Average PSF,Last Week Total Dollar Volume
0,68,700760,495,47651700.0


In [26]:
last_week['Last Week Total Dollar Volume'].iloc[0]

47651700.0

In [27]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           75             1133152          648                     84986400.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [28]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [29]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [30]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [31]:
pd.set_option('display.max_columns',None)

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-10,Condo/Co-op,8955 Collins Ave #702,Surfside,FL,33154.0,12250000.0,3.0,3.5,NORMANDY BEACH 2ND AMENDE,3136.0,NaN,2020.0,NaN,3906.0,11796.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8955-Collin...,MARMLS,A11787450,N,Y,25.875917,-80.121106,https://www.redfin.com/FL/Surfside/8955-Collin...,Miltiadis Kastanis,"Compass Florida, LLC.",Pablo Alfaro,"Compass Florida, LLC.",Devin Kay,Douglas Elliman,None,None,1,orange


In [33]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [34]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-10,Condo/Co-op,8955 Collins Ave #702,Surfside,FL,33154.0,12250000.0,3.0,3.5,NORMANDY BEACH 2ND AMENDE,3136.0,NaN,2020.0,NaN,3906.0,11796.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8955-Collin...,MARMLS,A11787450,N,Y,25.875917,-80.121106,https://www.redfin.com/FL/Surfside/8955-Collin...,Miltiadis Kastanis,"Compass Florida, LLC.",Pablo Alfaro,"Compass Florida, LLC.",Devin Kay,Douglas Elliman,None,None,1,orange
1,PAST SALE,2025-07-11,Condo/Co-op,17975 Collins Ave #3102,Sunny Isles Beach,FL,33160.0,8900000.0,4.0,4.5,THE ESTATES ACQUALINA,4385.0,NaN,2022.0,NaN,2030.0,9029.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11735146,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adriana Vargas Hernandez,Acqualina Realty,None,None,Ashley Cusack,BHHS EWM Realty,Nancy Batchelor,"Compass Florida, LLC.",2,blue
2,PAST SALE,2025-07-10,Condo/Co-op,10201 Collins Ave #2306,Bal Harbour,FL,33154.0,6575000.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2505.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11678342,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Anna Sherrill,One Sotheby's International Re,Adriano Biasella,One Sotheby's International Re,Ethan Assouline,Douglas Elliman,None,None,3,blue
3,PAST SALE,2025-07-10,Condo/Co-op,300 Sunny Isles Blvd Unit 4-502,Sunny Isles Beach,FL,33160.0,4700000.0,6.0,5.5,PARQUE TOWERS CONDO,4650.0,NaN,2019.0,NaN,1011.0,2934.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11787264,N,Y,25.929383,-80.125611,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,Eyal Chai,Residential Realty Group Inc,None,None,Jessica Sonino,LoKation,None,None,4,blue
4,PAST SALE,2025-07-09,Condo/Co-op,17141 Collins Ave #701,Sunny Isles Beach,FL,33160.0,4700000.0,3.0,3.5,MUSE CONDO,3195.0,NaN,2018.0,NaN,1471.0,9178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11744934,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adam Ziefer,The Corcoran Group,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue
5,PAST SALE,2025-07-11,Condo/Co-op,445 Grand Bay Dr #1210,Key Biscayne,FL,33149.0,4500000.0,3.0,4.0,GRAND BAY RESIDENCES COND,2950.0,NaN,1996.0,NaN,1525.0,3679.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/445-Gra...,MARMLS,A11759960,N,Y,25.692619,-80.158994,https://www.redfin.com/FL/Key-Biscayne/445-Gra...,Adolfo Malave,BHHS EWM Realty,None,None,Maria Vial-Ducaud,BHHS EWM Realty,None,None,6,blue
6,PAST SALE,2025-07-08,Condo/Co-op,500 Alton #4304,Miami Beach,FL,33139.0,3890000.0,2.0,2.5,FIVE PARK,1434.0,NaN,2024.0,NaN,2713.0,3477.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11765026,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,None,None,Angelica Garcia,Douglas Elliman,None,None,7,blue
7,PAST SALE,2025-07-10,Condo/Co-op,9801 Collins Ave Unit 6A Oceanfront,Bal Harbour,FL,33154.0,2375000.0,2.0,2.5,BALMORAL CONDO,1983.0,NaN,1977.0,NaN,1198.0,2743.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11794859,N,Y,25.890231,-80.122819,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,Silvia Olivera,"FIP Realty Services, LLC",None,None,Christian Espinosa,Optimar International Realty,None,None,8,blue
8,PAST SALE,2025-07-08,Condo/Co-op,2831 S Bayshore Dr #1701,Miami,FL,33133.0,2350000.0,3.0,2.5,CLUB RESIDENCES AT PARK G,1281.0,NaN,2018.0,NaN,1835.0,3505.0,Sold,

In [35]:
%store -r map_box_api_key

In [36]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [37]:
m.save('index.html')

## Data snagger

In [38]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [39]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-07-10,Condo/Co-op,8955 Collins Ave #702,Surfside,FL,33154.0,12250000.0,3.0,3.5,NORMANDY BEACH 2ND AMENDE,3136.0,NaN,2020.0,NaN,3906.0,11796.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8955-Collin...,MARMLS,A11787450,N,Y,25.875917,-80.121106,https://www.redfin.com/FL/Surfside/8955-Collin...,Miltiadis Kastanis,"Compass Florida, LLC.",Pablo Alfaro,"Compass Florida, LLC.",Devin Kay,Douglas Elliman,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [40]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_07142025


# CREATE TEMPLATE 

In [41]:
muni_set = set(df_top_ten['CITY'])

In [42]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [43]:
df_top_ten.reset_index(inplace=True,drop=True)

In [44]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [45]:
top_sale

'NORMANDY BEACH 2ND AMENDE at 8955 Collins Ave #702 in Surfside'

In [46]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [47]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-10,Condo/Co-op,8955 Collins Ave #702,Surfside,FL,33154.0,12250000.0,3.0,3.5,NORMANDY BEACH 2ND AMENDE,3136.0,NaN,2020.0,NaN,3906.0,11796.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8955-Collin...,MARMLS,A11787450,N,Y,25.875917,-80.121106,https://www.redfin.com/FL/Surfside/8955-Collin...,Miltiadis Kastanis,"Compass Florida, LLC.",Pablo Alfaro,"Compass Florida, LLC.",Devin Kay,Douglas Elliman,None,None,1,orange,NORMANDY BEACH 2ND AMENDE at 8955 Collins Ave ...
1,PAST SALE,2025-07-11,Condo/Co-op,17975 Collins Ave #3102,Sunny Isles Beach,FL,33160.0,8900000.0,4.0,4.5,THE ESTATES ACQUALINA,4385.0,NaN,2022.0,NaN,2030.0,9029.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11735146,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adriana Vargas Hernandez,Acqualina Realty,None,None,Ashley Cusack,BHHS EWM Realty,Nancy Batchelor,"Compass Florida, LLC.",2,blue,THE ESTATES ACQUALINA at 17975 Collins Ave #31...
2,PAST SALE,2025-07-10,Condo/Co-op,10201 Collins Ave #2306,Bal Harbour,FL,33154.0,6575000.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2505.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11678342,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Anna Sherrill,One Sotheby's International Re,Adriano Biasella,One Sotheby's International Re,Ethan Assouline,Douglas Elliman,None,None,3,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
3,PAST SALE,2025-07-10,Condo/Co-op,300 Sunny Isles Blvd Unit 4-502,Sunny Isles Beach,FL,33160.0,4700000.0,6.0,5.5,PARQUE TOWERS CONDO,4650.0,NaN,2019.0,NaN,1011.0,2934.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11787264,N,Y,25.929383,-80.125611,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,Eyal Chai,Residential Realty Group Inc,None,None,Jessica Sonino,LoKation,None,None,4,blue,PARQUE TOWERS CONDO at 300 Sunny Isles Blvd Un...
4,PAST SALE,2025-07-09,Condo/Co-op,17141 Collins Ave #701,Sunny Isles Beach,FL,33160.0,4700000.0,3.0,3.5,MUSE CONDO,3195.0,NaN,2018.0,NaN,1471.0,9178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11744934,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adam Ziefer,The Corcoran Group,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue,MUSE CONDO at 17141 Collins Ave #701 in Sunny ...
5,PAST SALE,2025-07-11,Condo/Co-op,445 Grand Bay Dr #1210,Key Biscayne,FL,33149.0,4500000.0,3.0,4.0,GRAND BAY RESIDENCES COND,2950.0,NaN,1996.0,NaN,1525.0,3679.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/445-Gra...,MARMLS,A11759960,N,Y,25.692619,-80.158994,https://www.redfin.com/FL/Key-Biscayne/445-Gra...,Adolfo Malave,BHHS EWM Realty,None,None,Maria Vial-Ducaud,BHHS EWM Realty,None,None,6,blue,GRAND BAY RESIDENCES COND at 445 Grand Bay Dr ...
6,PAST SALE,2025-07-08,Condo/Co-op,500 Alton #4304,Miami Beach,FL,33139.0,3890000.0,2.0,2.5,FIVE PARK,1434.0,NaN,2024.0,NaN,2713.0,3477.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11765026,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,None,None,Angelica Garcia,Douglas Elliman,None,None,7,blue,FIVE PARK at 500 Alton #4304 in Miami Beach
7,PAST SALE,2025-07-10,Condo/Co-op,9801 Collins Ave Unit 6A Oceanfront,Bal Harbour,FL,33154.0,2375000.0,2.0,2.5,BALMORAL CONDO,1983.0,NaN,1977.0,NaN,1198.0,2743.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11794859,

In [49]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: NORMANDY BEACH 2ND AMENDE closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,900,000 to $12,250,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Surfside, Miami Beach, Bal Harbour, Sunny Isles Beach, Key Biscayne, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 75 condo sales totaling $84,986,400 million from July 7 to July 13. The previous week, brokers closed 68 condo sales totaling $47,651,700.

Last week’s units sold for an average of $1,133,152, higher than the $700,7

In [50]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [51]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [52]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-10,Condo/Co-op,8955 Collins Ave #702,Surfside,FL,33154.0,12250000.0,3.0,3.5,NORMANDY BEACH 2ND AMENDE,3136.0,NaN,2020.0,NaN,3906.0,11796.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8955-Collin...,MARMLS,A11787450,N,Y,25.875917,-80.121106,https://www.redfin.com/FL/Surfside/8955-Collin...,Miltiadis Kastanis,"Compass Florida, LLC.",Pablo Alfaro,"Compass Florida, LLC.",Devin Kay,Douglas Elliman,None,None,1,orange,NORMANDY BEACH 2ND AMENDE at 8955 Collins Ave ...
1,PAST SALE,2025-07-11,Condo/Co-op,17975 Collins Ave #3102,Sunny Isles Beach,FL,33160.0,8900000.0,4.0,4.5,THE ESTATES ACQUALINA,4385.0,NaN,2022.0,NaN,2030.0,9029.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11735146,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adriana Vargas Hernandez,Acqualina Realty,None,None,Ashley Cusack,BHHS EWM Realty,Nancy Batchelor,"Compass Florida, LLC.",2,blue,THE ESTATES ACQUALINA at 17975 Collins Ave #31...
2,PAST SALE,2025-07-10,Condo/Co-op,10201 Collins Ave #2306,Bal Harbour,FL,33154.0,6575000.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2505.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11678342,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Anna Sherrill,One Sotheby's International Re,Adriano Biasella,One Sotheby's International Re,Ethan Assouline,Douglas Elliman,None,None,3,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
3,PAST SALE,2025-07-10,Condo/Co-op,300 Sunny Isles Blvd Unit 4-502,Sunny Isles Beach,FL,33160.0,4700000.0,6.0,5.5,PARQUE TOWERS CONDO,4650.0,NaN,2019.0,NaN,1011.0,2934.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11787264,N,Y,25.929383,-80.125611,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,Eyal Chai,Residential Realty Group Inc,None,None,Jessica Sonino,LoKation,None,None,4,blue,PARQUE TOWERS CONDO at 300 Sunny Isles Blvd Un...
4,PAST SALE,2025-07-09,Condo/Co-op,17141 Collins Ave #701,Sunny Isles Beach,FL,33160.0,4700000.0,3.0,3.5,MUSE CONDO,3195.0,NaN,2018.0,NaN,1471.0,9178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11744934,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adam Ziefer,The Corcoran Group,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue,MUSE CONDO at 17141 Collins Ave #701 in Sunny ...
5,PAST SALE,2025-07-11,Condo/Co-op,445 Grand Bay Dr #1210,Key Biscayne,FL,33149.0,4500000.0,3.0,4.0,GRAND BAY RESIDENCES COND,2950.0,NaN,1996.0,NaN,1525.0,3679.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/445-Gra...,MARMLS,A11759960,N,Y,25.692619,-80.158994,https://www.redfin.com/FL/Key-Biscayne/445-Gra...,Adolfo Malave,BHHS EWM Realty,None,None,Maria Vial-Ducaud,BHHS EWM Realty,None,None,6,blue,GRAND BAY RESIDENCES COND at 445 Grand Bay Dr ...
6,PAST SALE,2025-07-08,Condo/Co-op,500 Alton #4304,Miami Beach,FL,33139.0,3890000.0,2.0,2.5,FIVE PARK,1434.0,NaN,2024.0,NaN,2713.0,3477.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11765026,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,None,None,Angelica Garcia,Douglas Elliman,None,None,7,blue,FIVE PARK at 500 Alton #4304 in Miami Beach
7,PAST SALE,2025-07-10,Condo/Co-op,9801 Collins Ave Unit 6A Oceanfront,Bal Harbour,FL,33154.0,2375000.0,2.0,2.5,BALMORAL CONDO,1983.0,NaN,1977.0,NaN,1198.0,2743.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11794859,

In [54]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: NORMANDY BEACH 2ND AMENDE closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,900,000 to $12,250,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Surfside, Miami Beach, Bal Harbour, Sunny Isles Beach, Key Biscayne, Miami, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 75 condo sales totaling $84,986,400 million from July 7 to July 13. The previous week, brokers closed 68 condo sales totaling $47,651,700.

Last week’s units sold for an average of $1,133,152, higher than the $700,7

In [55]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Surfside/8955-Collins-Ave-33154/unit-702/home/171043757


In [56]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17975-Collins-Ave-33160/unit-3102/home/194525976


In [57]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/North-Miami-Beach/15811-Collins-Ave-33160/unit-3302/home/43447688


In [58]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/2831-S-Bayshore-Dr-33133/unit-1701/home/79134161


In [59]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Surfside/8955-Collins-Ave-33154/unit-702/home/171043757


In [60]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/North-Miami-Beach/15811-Collins-Ave-33160/unit-3302/home/43447688


In [61]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-07-10,Condo/Co-op,8955 Collins Ave #702,Surfside,FL,33154.0,12250000.0,3.0,3.5,NORMANDY BEACH 2ND AMENDE,3136.0,NaN,2020.0,NaN,3906.0,11796.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/8955-Collin...,MARMLS,A11787450,N,Y,25.875917,-80.121106,https://www.redfin.com/FL/Surfside/8955-Collin...,Miltiadis Kastanis,"Compass Florida, LLC.",Pablo Alfaro,"Compass Florida, LLC.",Devin Kay,Douglas Elliman,None,None,1,orange,NORMANDY BEACH 2ND AMENDE at 8955 Collins Ave ...
1,PAST SALE,2025-07-11,Condo/Co-op,17975 Collins Ave #3102,Sunny Isles Beach,FL,33160.0,8900000.0,4.0,4.5,THE ESTATES ACQUALINA,4385.0,NaN,2022.0,NaN,2030.0,9029.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11735146,N,Y,25.942332,-80.120166,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adriana Vargas Hernandez,Acqualina Realty,None,None,Ashley Cusack,BHHS EWM Realty,Nancy Batchelor,"Compass Florida, LLC.",2,blue,THE ESTATES ACQUALINA at 17975 Collins Ave #31...
2,PAST SALE,2025-07-10,Condo/Co-op,10201 Collins Ave #2306,Bal Harbour,FL,33154.0,6575000.0,2.0,3.5,OCEANA BAL HARBOUR CONDO,2625.0,NaN,2016.0,NaN,2505.0,5360.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11678342,N,Y,25.895168,-80.123150,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,Anna Sherrill,One Sotheby's International Re,Adriano Biasella,One Sotheby's International Re,Ethan Assouline,Douglas Elliman,None,None,3,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
3,PAST SALE,2025-07-10,Condo/Co-op,300 Sunny Isles Blvd Unit 4-502,Sunny Isles Beach,FL,33160.0,4700000.0,6.0,5.5,PARQUE TOWERS CONDO,4650.0,NaN,2019.0,NaN,1011.0,2934.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,MARMLS,A11787264,N,Y,25.929383,-80.125611,https://www.redfin.com/FL/Sunny-Isles-Beach/30...,Eyal Chai,Residential Realty Group Inc,None,None,Jessica Sonino,LoKation,None,None,4,blue,PARQUE TOWERS CONDO at 300 Sunny Isles Blvd Un...
4,PAST SALE,2025-07-09,Condo/Co-op,17141 Collins Ave #701,Sunny Isles Beach,FL,33160.0,4700000.0,3.0,3.5,MUSE CONDO,3195.0,NaN,2018.0,NaN,1471.0,9178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11744934,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Adam Ziefer,The Corcoran Group,None,None,NON-MLS MEMBER,MAR NON MLS MEMBER,None,None,5,blue,MUSE CONDO at 17141 Collins Ave #701 in Sunny ...
5,PAST SALE,2025-07-11,Condo/Co-op,445 Grand Bay Dr #1210,Key Biscayne,FL,33149.0,4500000.0,3.0,4.0,GRAND BAY RESIDENCES COND,2950.0,NaN,1996.0,NaN,1525.0,3679.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/445-Gra...,MARMLS,A11759960,N,Y,25.692619,-80.158994,https://www.redfin.com/FL/Key-Biscayne/445-Gra...,Adolfo Malave,BHHS EWM Realty,None,None,Maria Vial-Ducaud,BHHS EWM Realty,None,None,6,blue,GRAND BAY RESIDENCES COND at 445 Grand Bay Dr ...
6,PAST SALE,2025-07-08,Condo/Co-op,500 Alton #4304,Miami Beach,FL,33139.0,3890000.0,2.0,2.5,FIVE PARK,1434.0,NaN,2024.0,NaN,2713.0,3477.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/500-Alto...,MARMLS,A11765026,N,Y,25.775127,-80.141257,https://www.redfin.com/FL/Miami-Beach/500-Alto...,Anca Mirescu,Douglas Elliman,None,None,Angelica Garcia,Douglas Elliman,None,None,7,blue,FIVE PARK at 500 Alton #4304 in Miami Beach
7,PAST SALE,2025-07-10,Condo/Co-op,9801 Collins Ave Unit 6A Oceanfront,Bal Harbour,FL,33154.0,2375000.0,2.0,2.5,BALMORAL CONDO,1983.0,NaN,1977.0,NaN,1198.0,2743.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11794859,

## Time on Market Calculator

In [63]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 1, 30) ## List (Earlier) date
date2 = datetime(2025, 7, 11) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

162


## Clean CSV for Datawrapper Chart

In [64]:
chart_df = df_top_ten

In [65]:
chart_df = chart_df.fillna(" ")

In [66]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [67]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [68]:
chart_df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME', 'LISTING AGENTS',
       'BUYING AGENTS'],
      dtype='object')

In [69]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [70]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [71]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [72]:
chart_df['PPSF'].astype(float)

0    3906.0
1    2030.0
2    2505.0
3    1011.0
4    1471.0
5    1525.0
6    2713.0
7    1198.0
8    1835.0
9     963.0
Name: PPSF, dtype: float64

In [73]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [74]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    'PPSF',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [75]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-07-10,NORMANDY BEACH 2ND AMENDE,8955 Collins Ave #702 in Surfside,"$12,250,000","3,136","$3,906",2020,"Miltiadis Kastanis Compass Florida, LLC. Pablo...",Devin Kay Douglas Elliman
1,2025-07-11,THE ESTATES ACQUALINA,17975 Collins Ave #3102 in Sunny Isles Beach,"$8,900,000","4,385","$2,030",2022,Adriana Vargas Hernandez Acqualina Realty,Ashley Cusack BHHS EWM Realty Nancy Batchelor ...
2,2025-07-10,OCEANA BAL HARBOUR CONDO,10201 Collins Ave #2306 in Bal Harbour,"$6,575,000","2,625","$2,505",2016,Anna Sherrill One Sotheby's International Re A...,Ethan Assouline Douglas Elliman
3,2025-07-10,PARQUE TOWERS CONDO,300 Sunny Isles Blvd Unit 4-502 in Sunny Isles...,"$4,700,000","4,650","$1,011",2019,Eyal Chai Residential Realty Group Inc,Jessica Sonino LoKation
4,2025-07-09,MUSE CONDO,17141 Collins Ave #701 in Sunny Isles Beach,"$4,700,000","3,195","$1,471",2018,Adam Ziefer The Corcoran Group,NON-MLS MEMBER MAR NON MLS MEMBER
5,2025-07-11,GRAND BAY RESIDENCES COND,445 Grand Bay Dr #1210 in Key Biscayne,"$4,500,000","2,950","$1,525",1996,Adolfo Malave BHHS EWM Realty,Maria Vial-Ducaud BHHS EWM Realty
6,2025-07-08,FIVE PARK,500 Alton #4304 in Miami Beach,"$3,890,000","1,434","$2,713",2024,Anca Mirescu Douglas Elliman,Angelica Garcia Douglas Elliman
7,2025-07-10,BALMORAL CONDO,9801 Collins Ave Unit 6A Oceanfront in Bal Har...,"$2,375,000","1,983","$1,198",1977,"Silvia Olivera FIP Realty Services, LLC",Christian Espinosa Optimar International Realt...
8,2025-07-08,CLUB RESIDENCES AT PARK G,2831 S Bayshore Dr #1701 in Miami,"$2,350,000","1,281","$1,835",2018,"Francis Pimentel de Matos Compass Florida, LLC...","Sarahy Yanez Compass Florida, LLC Riley Smith ..."
9,2025-07-10,TDR TOWER III CONDO,15811 Collins Ave #3302 in Sunny Isles Beach,"$1,900,000","1,974",$963,2010,Relindes Snak Brickss Real Estate Company,Adam Kruszelnicki One Sotheby's International ...


In [76]:
csv_date_string = today.strftime("%m_%d_%Y")

In [77]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [78]:
chart_df.columns

Index(['SOLD DATE', 'LOCATION', 'ADDRESS', 'PRICE', 'SQUARE FEET', 'PPSF',
       'YEAR BUILT', 'LISTING AGENTS', 'BUYING AGENTS'],
      dtype='object')